In [19]:
import psycopg2
import pandas as pd
from jinja2 import Template

In [20]:
# Establish a connection
conn = psycopg2.connect(
    database="numeric_top_10_features",
    user="postgres",
    password="1234",
    host="localhost"
)

In [21]:
query = "SELECT * FROM final_table;"
df = pd.read_sql(query, conn)

/var/folders/nx/7gj5ddwj3qq_yn5v4hkglrcm0000gq/T/ipykernel_12725/176675782.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [33]:
# Exclude non-numeric columns from the DataFrame
numeric_df = df.select_dtypes(include=['number'])

# Render the template
report_template = """
<h1>Report on Fraud Analysis</h1>
<p>Summary of the dataset:</p>
{{ df.head().to_html() }}

<h2>Fraud Distribution Count:</h2>
{{ fraud_distribution_count|safe }}

<h2>Numerical Features:</h2>
{{ numerical_features|safe }}

<h2>Correlation Matrix:</h2>
{{ correlation_matrix|safe }}
"""

template = Template(report_template)

fraud_distribution_count = df['FraudFound_P'].value_counts().to_frame().to_html()
numerical_features = numeric_df.columns.to_list()
correlation_matrix = numeric_df.corr().to_html()

report_html = template.render(df=df, fraud_distribution_count=fraud_distribution_count, numerical_features=numerical_features, correlation_matrix=correlation_matrix)

# Save the report to an HTML file
with open("Numeric Data Report.html", "w") as f:
    f.write(report_html)

In [23]:
conn.close()